<a href="https://colab.research.google.com/github/fermasia/infovis/blob/gh-pages/Charts_COVID_Altair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
# Importar dataset con info COVID Argentina
df = pd.read_csv('https://raw.githubusercontent.com/fermasia/infovis/gh-pages/final_prov.csv')
# Importar Metadata de Provincias
provincias = pd.read_csv('https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.7/download/provincias.csv')

In [ ]:
# Prepación inicial de información para graficar
df['mes'] = pd.to_datetime(df.fecha).dt.strftime('%Y-%m') #Agregar mes-año al dataset
df = df[df.fecha >= df.fecha[df['cuenta_muertes'] > 0].min()] #filtrar fecha desde el 1er fallecimiento
df['vac'] = df['1'] + df['2'] # sumarizar 1ras y 2das dosis

In [ ]:
# DATASET para Mapa Choropleth de Muertes y Vacunaciones al dia de hoy
##
muervac_hoy = df.groupby(['residencia_provincia_id'])[['cuenta_muertes','1']].sum().reset_index()
muervac_hoy.rename(columns={'residencia_provincia_id':'id'},inplace=True)
muervac_hoy = pd.merge(muervac_hoy,provincias,how='inner',on='id')
pobla = df.groupby(['residencia_provincia_id'])['Poblacion'].max().reset_index().rename(columns={'residencia_provincia_id':'id'})
muervac_hoy = pd.merge(muervac_hoy,pobla,how='left',on='id')
muervac_hoy['ratio_vac'] = (muervac_hoy['1'] / muervac_hoy['Poblacion']).round(2)
muervac_hoy.to_csv('muervac.csv',index=False)
# dump del archivo
# files.download('muervac.csv')

In [ ]:
# DATASET para Mapa Choropleth de Muertes y Vacunaciones hace 3 meses
##
df[df.fecha < '2021-05-01']
muervac_3m = df[df.fecha < '2021-05-01'].groupby(['residencia_provincia_id'])[['cuenta_muertes','1']].sum().reset_index()
muervac_3m.rename(columns={'residencia_provincia_id':'id'},inplace=True)
muervac_3m = pd.merge(muervac_3m,provincias,how='inner',on='id')
pobla = df.groupby(['residencia_provincia_id'])['Poblacion'].max().reset_index().rename(columns={'residencia_provincia_id':'id'})
muervac_3m = pd.merge(muervac_3m,pobla,how='left',on='id')
muervac_3m['ratio_vac'] = (muervac_3m['1'] / muervac_3m['Poblacion']).round(2)
#muervac_3m.to_csv('muervac_3m.csv',index=False)
#files.download('muervac_3m.csv')

In [ ]:
# Prearar datos para HEATMAP - Vacunas dosis 2
#df.groupby(['residencia_provincia_id','fecha'])[['cuenta_muertes','vac']].sum().reset_index()
muervac_mes_2d = df.groupby(['residencia_provincia_id','mes'])[['cuenta_muertes','2']].sum().reset_index()
muervac_mes_2d.rename(columns={'residencia_provincia_id':'id'},inplace=True)
muervac_mes_2d = pd.merge( muervac_mes_2d,provincias,how='inner',on='id')
pobla = df.groupby(['residencia_provincia_id'])['Poblacion'].max().reset_index().rename(columns={'residencia_provincia_id':'id'})
muervac_mes_2d = pd.merge( muervac_mes_2d,pobla,how='left',on='id')
muervac_mes_2d['cumsum'] =  muervac_mes_2d[['mes','id','2']].sort_values(by='mes').groupby(['id']).cumsum(skipna=False)
muervac_mes_2d['ratio_vac'] = ( muervac_mes_2d['cumsum'] /  muervac_mes_2d['Poblacion']).round(2)
muervac_mes_2d.replace({'Ciudad Autónoma de Buenos Aires':'CABA','Tierra del Fuego, Antártida e Islas del Atlántico Sur':'Tierra del Fuego'},inplace=True)
# muervac_mes_2d.to_csv(' muervac_mes_2d.csv',index=False)
# files.download(' muervac_mes_2d.csv')

In [ ]:
# Prearar datos para HEATMAP - Vacunas dosis 1
#df.groupby(['residencia_provincia_id','fecha'])[['cuenta_muertes','vac']].sum().reset_index()
muervac_mes = df.groupby(['residencia_provincia_id','mes'])[['cuenta_muertes','1']].sum().reset_index()
muervac_mes.rename(columns={'residencia_provincia_id':'id'},inplace=True)
muervac_mes = pd.merge(muervac_mes,provincias,how='inner',on='id')
pobla = df.groupby(['residencia_provincia_id'])['Poblacion'].max().reset_index().rename(columns={'residencia_provincia_id':'id'})
muervac_mes = pd.merge(muervac_mes,pobla,how='left',on='id')
muervac_mes['cumsum'] = muervac_mes[['mes','id','1']].sort_values(by='mes').groupby(['id']).cumsum(skipna=False)
muervac_mes['ratio_vac'] = (muervac_mes['cumsum'] / muervac_mes['Poblacion']).round(2)
muervac_mes.replace({'Ciudad Autónoma de Buenos Aires':'CABA','Tierra del Fuego, Antártida e Islas del Atlántico Sur':'Tierra del Fuego'},inplace=True)
#muervac_mes.to_csv('muervac_mes.csv',index=False)
#files.download('muervac_mes.csv')

In [ ]:
# Preparar información para graficar
df['mes'] = pd.to_datetime(df.fecha).dt.strftime('%Y-%m') #Agregar mes
df = df[df.fecha >= df.fecha[df['cuenta_muertes'] > 0].min()] #filtrar fecha
df['vac'] = df['1'] + df['2'] # sumarizar 1ras y 2das dosis

#Crear Dataset de Vac & Falle
vacfalle = df.groupby('fecha').sum()[['vac','cuenta_muertes']].reset_index().fillna(0)
vacfalle[['falle_roll','vac_roll']] = vacfalle[['cuenta_muertes','vac']].rolling(window=7).mean().fillna(0).round(0) # Agregar media móvil
vacfalle.drop(columns=['vac','cuenta_muertes'],inplace=True) # descartar columnas
vacfalle.rename(columns={'falle_roll':'Fallecidos','vac_roll':"Vacunados",'fecha':'Fecha','cuenta_diag':'Contagios'},inplace=True) #Renombrar Columnas

# Crear Dataset de Vac y Contagiados
contavac = df.groupby('fecha').sum()[['1','cuenta_diag']].reset_index().fillna(0)
contavac[['vac_roll','conta_roll']] = contavac.rolling(window=7).mean().round(0) # Agregar media movil
contavac.rename(columns={'conta_roll':'Contagios','vac_roll':"Vacunados",'fecha':'Fecha'},inplace=True) #Renombrar Columnas

In [ ]:
# GRAFICO 1

import altair as alt
from vega_datasets import data
from urllib import request
import json

# fetch & enable a Spanish timeFormat locale.
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-ES.json') as f:
  es_time_format = json.load(f)
alt.renderers.set_embed_options(timeFormatLocale=es_time_format)

source = vacfalle

base = alt.Chart(source,title='Fallecidos y Vacunados por día - media móvil últimos 7 días').transform_calculate(
    Muertes="'Muertes'",
    Vacunas="'Vacunas'",
)
scale = alt.Scale(domain=["Muertes","Vacunas"], range=['red','#026b83'])


falle = base.mark_area(opacity=0.25).encode(
    alt.X("Fecha:T",axis=alt.Axis(title='')),
    alt.Y("Fallecidos:Q",scale=alt.Scale(domain=(0, 1000)),axis=alt.Axis(title='Fallecidos por día'))
    ,color=alt.Color('Muertes:N', title='')
    ,tooltip=['Fecha:T','Fallecidos','Vacunados']
).properties(
    width=1000,
    height=400
)

vacu = base.mark_area(opacity=0.45).encode(
    alt.X("Fecha:T"),
    alt.Y("Vacunados:Q",axis=alt.Axis(title='Vacunas por día'))
    ,color=alt.Color('Vacunas:N', scale=scale, title='')
    ,tooltip=['Fecha:T', 'Fallecidos','Vacunados']
)

alt.layer(falle,vacu).resolve_scale(
    y = 'independent'
).properties(
    width=1000,
    height=400
)

#chart1.save('Fallecidos.html')
#files.download('Fallecidos.html')

alt.LayerChart(...)

In [ ]:
# GRAFICO 2

import altair as alt
from vega_datasets import data
from urllib import request
import json

# fetch & enable a Spanish timeFormat locale.
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-ES.json') as f:
  es_time_format = json.load(f)
alt.renderers.set_embed_options(timeFormatLocale=es_time_format)

source = contavac

base = alt.Chart(source,title='Contagiados y Vacunados por día - media móvil últimos 7 días').transform_calculate(
    Contagiados="'Contagiados'",
    Vacunas="'Vacunas'",
)

scale = alt.Scale(domain=["Contagiados","Vacunas"], range=['#ff742e','#026b83'])


conta = base.mark_area(opacity=0.45).encode(
    alt.X("Fecha:T",axis=alt.Axis(title='')),
    alt.Y("Contagios:Q",scale=alt.Scale(domain=(0, 50000)),axis=alt.Axis(title='Contagiados por día'))
    ,color=alt.Color('Contagiados:N', scale=scale, title='')
    ,tooltip=['Fecha:T','Contagios','Vacunados']
).properties(
    width=1000,
    height=400
)

vacu = base.mark_area(opacity=0.45).encode(
    alt.X("Fecha:T"),
    alt.Y("Vacunados:Q",axis=alt.Axis(title='Vacunas por día'))
    ,color=alt.Color('Vacunas:N', scale=scale, title='')
    ,tooltip=['Fecha:T','Contagios','Vacunados']
)

alt.layer(conta,vacu).resolve_scale(
    y = 'independent'
).properties(
    width=1000,
    height=400
)

#chart2.save('Contagiados.html')
#files.download('Contagiados.html')

alt.LayerChart(...)

In [ ]:
import altair as alt
from vega_datasets import data

orden_provs = muervac_mes[['nombre','ratio_vac']].groupby('nombre').max().reset_index().sort_values(by='ratio_vac',ascending=False)['nombre'].to_list()
orden_provs2 = muervac_mes_2d[['nombre','ratio_vac']].groupby('nombre').max().reset_index().sort_values(by='ratio_vac',ascending=False)['nombre'].to_list()

source = muervac_mes[['mes','nombre','ratio_vac']]#[muervac_mes.mes > '2020-3']
source2 = muervac_mes_2d[['mes','nombre','ratio_vac']]#[muervac_mes.mes > '2020-7']

graf2 = alt.Chart(source2,title='Ratio Población Vacunada - 2da Dosis').mark_rect(opacity=0.85).encode(
    alt.X('mes:O', title=''),
    alt.Y('nombre:O',sort=orden_provs2,title=''),
    color=alt.Color('ratio_vac:Q', scale=alt.Scale(domain=[0,0.85],scheme='redyellowgreen'),title=''),
    tooltip=[{"type": "nominal", "field": "nombre","title":"Provincia"},
             {"type": "quantitative", "field": "ratio_vac","title":"Ratio"}]
).properties(
    height=500,
    width=1000
)

graf1 = alt.Chart(source,title='Ratio Población Vacunada - 1ra Dosis').mark_rect(opacity=0.85).encode(
    alt.X('mes:O', title=''),
    alt.Y('nombre:O',sort=orden_provs,title=''),
    color=alt.Color('ratio_vac:Q', scale=alt.Scale(domain=[0, 0.85],scheme='redyellowgreen'),title=''),
    tooltip=[{"type": "nominal", "field": "nombre","title":"Provincia"},
             {"type": "quantitative", "field": "ratio_vac","title":"Ratio"}]
).properties(
    height=500,
    width=1000
)

#graf = graf1 | graf2

final = alt.vconcat(graf1,graf2
).resolve_scale(color='independent'
).configure_axis(
    labelFontSize=11,
    titleFontSize=20
).configure_legend(
    cornerRadius=1,
    gradientThickness=14,
    labelFontSize=10
).configure_title(
    fontSize=16
).properties().configure_axisX(labelAngle=0)

final
#final.save('final.html')
#files.download('final.html')

alt.VConcatChart(...)